# House Prices - Predição

Após realizar uma análise exploratória de dados detalhada e filtrar o máximo possível, sempre na intenção de conter o máximo de informação útil possível.

Agora, vamos realizar uma bateria de modelos de predição com os dados que filtrei.

## Inicialização

Puxando as bibliotecas necessárias para a realização do relatório e do banco de dados.

In [1]:
# bibliotecas
import numpy as np
import pandas as pd
import random as rd
import sklearn.metrics as metrics

from sklearn. model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# banco de dados

# Dados
dados = pd.read_csv("./dados_tratados.csv")

Verificando a integridade dos dados

In [2]:
dados

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,1,0,Reg,Lvl,AllPub,Inside,...,0,0,0.0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,1,0,Reg,Lvl,AllPub,FR2,...,0,0,0.0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,1,0,IR1,Lvl,AllPub,Inside,...,0,0,0.0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,1,0,IR1,Lvl,AllPub,Corner,...,0,0,0.0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,1,0,IR1,Lvl,AllPub,FR2,...,0,0,0.0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442,60,RL,62.0,7917,1,0,Reg,Lvl,AllPub,Inside,...,0,0,0.0,0,0,8,2007,WD,Normal,175000
1443,20,RL,85.0,13175,1,0,Reg,Lvl,AllPub,Inside,...,0,0,0.0,0,0,2,2010,WD,Normal,210000
1444,70,RL,66.0,9042,1,0,Reg,Lvl,AllPub,Inside,...,0,0,0.0,1,2500,5,2010,WD,Normal,266500
1445,20,RL,68.0,9717,1,0,Reg,Lvl,AllPub,Inside,...,0,0,0.0,0,0,4,2010,WD,Normal,142125


### Separando os dados de treino e teste

Os dados presente ja fazem parte de uma separação treino/teste. Contudo vamos separar novamente para averiguar a performance dos modelos testados. Vamos também selecionar apenas as covariáveis que impactam diretamente na variável resposta, estas foram definidas no relatório de análise exploratória e tratamento desses dados.

In [3]:
# fixando uma seed para reprodutibilidade dos testes
rd.seed(10)

# existem apenas 8 colunas de interesse que possuem valores numéricos + variável resp.
numerics = ['OverallQual', 
            'GrLivArea',
            'ExterQual',
            'KitchenQual',
            'GarageArea', 
            'GarageCars', 
            'TotalBsmtSF',
            '1stFlrSF',
           'SalePrice']

# inserindo essas colunas em um novo dataframe
novo_dados = dados[numerics]

Vamos agora "*Dummieficar*" as variáveis categóricas, para que possamos trabalhar em um modelo de regressão de forma precisa. 

In [4]:
# selecionando colunas com variaveis categoricas
idx = (dados.applymap(type) == str).all(0) 

# inserindo as variaveis categoricas em um banco de dados auxiliar
df_new = dados[dados.columns[idx]]
df_new = pd.get_dummies(df_new)

# Inserindo as variáveis dummieficadas em nosso banco original
novo_dados = novo_dados.join(df_new)

In [5]:
novo_dados

,OverallQual,GrLivArea,ExterQual,KitchenQual,GarageArea,GarageCars,TotalBsmtSF,1stFlrSF,SalePrice,MSZoning_C (all),...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,7,1710,4,4,548,2,856,856,208500,0,...,0,0,0,1,0,0,0,0,1,0
1,6,1262,3,3,460,2,1262,1262,181500,0,...,0,0,0,1,0,0,0,0,1,0
2,7,1786,4,4,608,2,920,920,223500,0,...,0,0,0,1,0,0,0,0,1,0
3,7,1717,3,4,642,3,756,961,140000,0,...,0,0,0,1,1,0,0,0,0,0
4,8,2198,4,4,836,3,1145,1145,250000,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442,6,1647,3,3,460,2,953,953,175000,0,...,0,0,0,1,0,0,0,0,1,0
1443,6,2073,3,3,500,2,1542,2073,210000,0,...,0,0,0,1,0,0,0,0,1,0
1444,7,2340,5,4,252,1,1152,1188,266500,0,...,0,0,0,1,0,0,0,0,1,0
1445,5,1078,3,4,240,1,1078,1078,142125,0,...,0,0,0,1,0,0,0,0,1,0


In [6]:
# selecionando as colunas com covariáveis
X = novo_dados.loc[:, novo_dados.columns != 'SalePrice']

# selecionando a coluna com a variável resposta
y = novo_dados.loc[:, novo_dados.columns == 'SalePrice']

# separando os dados entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
y

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1442,175000
1443,210000
1444,266500
1445,142125


## Modelagem

Agora eu vou realizar uma bateria de modelagens para verificar quais modelos desempenham melhor com os dados que temos. Caso necessário seja, farei uma série de transformações e ajustes para melhorar a qualidade das predições.

Segue uma função que contém uma bateria de métricas para averiguar a qualidade dos modelos:

In [8]:
# função que analisa o desempenho do modelo
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse = metrics.mean_squared_error(y_true, y_pred) 
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    lista = [explained_variance, 
             r2, 
             mean_absolute_error, 
             mse,
             np.sqrt(mse)]
    
    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))
    
    return lista

### Regressão linear simples

In [9]:
modelo1 = LinearRegression()
modelo1.fit(X_train, y_train)

y_pred = modelo1.predict(X_test)

regression_results(y_test, y_pred)

explained_variance:  0.8433
r2:  0.8432
MAE:  20937.297
MSE:  947119920.8389
RMSE:  30775.3135


[0.8433099459498504,
 0.843164814498024,
 20937.2970205958,
 947119920.8388646,
 30775.313497003805]

### Random forest

Inicialmente vou realizar uma modelagem simples e direta para ver o desempenho do modelo

In [10]:
modelo2 = RandomForestClassifier()

modelo2.fit(X_train, y_train.values.ravel())

y_pred = modelo2.predict(X_test)

regression_results(y_test, y_pred)

explained_variance:  0.6696
r2:  0.6676
MAE:  27263.623
MSE:  2007139004.892
RMSE:  44801.105


[0.6695714385191067,
 0.667634466096256,
 27263.622988505747,
 2007139004.891954,
 44801.104951685666]

Podemos ver que não há uma melhora extremamente significativa em relação ao modelo de regressão simples. Entretanto podemos também melhorar esse desempenho ao mexer na hyperparametrização dele. Para isso Vamos visualizar em quais parametros podemos atualizar.

In [11]:
from pprint import pprint
pprint(modelo2.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


Vamos realizar uma busca aleatorizada dos melhores parâmetros para o nosso modelo. Isso servirá apenas como um formato de filtro para o nosso modelo final utilizando GridSearch

In [12]:
# Número de arvores na RF, de 200 em 200 até 1000
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]

# Escolhe o número máximo de variáveis 
max_features = ['auto', 'sqrt']

# Número máximo de divisões da arvore
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Mínimo de amostras a considerar para divisão da arvore
min_samples_split = [2, 5, 10]

# numero minimo de amostras das folhas
min_samples_leaf = [1, 2, 4]

# Método de seleção de amostras pra cada árvore
bootstrap = [True, False]

# Colocando os parametros em um formato legivel para a função
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [ ]:
# Procurando os melhores parâmetros de forma aleatorizada.
# Este processo demora um pouco por possuir 500 interações com o intuito de 
# buscar a melhor parametrização possível
modelo2_random = RandomizedSearchCV(estimator = modelo2, 
                                   param_distributions = random_grid, 
                                   n_iter = 500, 
                                   cv = 5, 
                                   verbose=2, 
                                   random_state=42, 
                                   n_jobs = -1)
# Fit the random search model
modelo2_random.fit(X_train, y_train.values.ravel())

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


C:\Users\013756631\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [ ]:
# Visualizando os melhores parametros escolhidos aleatoriamente
modelo2_random.best_params_

In [ ]:
y_pred = modelo2_random.predict(X_test)
regression_results(y_test, y_pred)

In [ ]:
# Criando uma parametrização do GrideSearchCV baseada na passada
param_grid = {
    'bootstrap': [True],
    'max_depth': [50, 60, 70, 80],
    'max_features': ['auto'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [5, 7, 9],
    'n_estimators': [100, 200, 300]
}

In [ ]:
grid_search = GridSearchCV(estimator = modelo2, 
                           param_grid = param_grid, 
                           cv = 5, 
                           n_jobs = -1, 
                           verbose = 2)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_


In [ ]:
y_pred = grid_search.predict(X_test)
regression_results(y_test, y_pred)

Após as escolhas de parametrização houve uma melhora pouco significativa do modelo com grid_search em comparação ao modelo comum.

### XGboosting

In [ ]:
modelo3 = XGBRegressor()

In [ ]:
modelo3.fit(X_train, y_train)
y_pred = modelo3.predict(X_test)
regression_results(y_test, y_pred)

É notável a diferença do XGboosting para os demais modelos. Vamos realizar a tunagem da hyperparametrização para tentar alcançar o máximo da eficácia do modelo. Vamos utilizar o GridSearch

In [ ]:
params = { 'max_depth': [7],
           'learning_rate': [0.001, 0.005],
           'n_estimators': [1500],
           'colsample_bytree': [0.35, 0.4, 0.45],
           'subsample': [0.15, 0.2, 0.25]}

In [ ]:
modelo3 = GridSearchCV(estimator=modelo3, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=2)

modelo3.fit(X_train, y_train)
print("Best parameters:", modelo3.best_params_)

Escolhidos os melhores parâmetros por meio de GridSearch. Vamos verificar o quanto foi efetivo essa tunagem dos hyperparametros.

In [ ]:
y_pred = modelo3.predict(X_test)
regression_results(y_test, y_pred)

Melhora significativa

### ADAboosting

In [ ]:
modelo4 = AdaBoostRegressor()
modelo4.fit(X_train, y_train)

In [ ]:
y_pred = modelo4.predict(X_test)
regression_results(y_test, y_pred)

In [ ]:
params = {'learning_rate': [0.1, 0.5, 1],
           'n_estimators': [500, 750,1000]}

modelo4 = GridSearchCV(estimator=modelo4, 
                       param_grid=params,)

modelo4.fit(X_train, y_train.values.ravel())
print("Best parameters:", modelo4.best_params_)

In [ ]:
y_pred = modelo4.predict(X_test)
regression_results(y_test, y_pred)

## Resultados prévios

Vamos analisar o comportamento de cada modelo de forma gráfica

In [ ]:
# atribuindo os resultados dos modelos
print("\n Modelo de regressão simples \n")
y_pred = modelo1.predict(X_test)
res1 = regression_results(y_test, y_pred)

print("\n Modelo de Random Forest \n")
y_pred = modelo2.predict(X_test)
res2 = regression_results(y_test, y_pred)

print("\n XGboosting \n")
y_pred = modelo3.predict(X_test)
res3 = regression_results(y_test, y_pred)

print("\n ADAboosting \n")
y_pred = modelo4.predict(X_test)
res4 = regression_results(y_test, y_pred)

Vamos inserir esses resultados em um banco de dados para que possamos fazer gráficos comparativos

In [ ]:
result = pd.DataFrame({"Reg. Simp." : res1[0:5],
                      "Rand. Forest": res2[0:5],
                      "XGboost": res3[0:5],
                      "ADAboost": res4[0:5]})
result 

In [ ]:
#plt.figure(figsize=(4, 5))
sns.countplot(x = )

## Transformações e melhorias possíveis

Explicar min max, normalização e PCA
fazer QQplot

### Normalizando e escalando

In [ ]:
from sklearn import preprocessing
scaler = MinMaxScaler()
# transform data

novo_dados[numerics] = scaler.fit_transform(novo_dados[numerics])
novo_dados[numerics] = preprocessing.normalize(novo_dados[numerics])

# selecionando as colunas com covariáveis
X = novo_dados.loc[:, novo_dados.columns != 'SalePrice']

# selecionando a coluna com a variável resposta
y = novo_dados.loc[:, novo_dados.columns == 'SalePrice']

# separando os dados entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
novo_dados

### Análise de Componentes Principais

In [ ]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.99)

In [ ]:
pca.fit(X_train)

In [ ]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
modelo3.fit(X_train, y_train.values.ravel())

In [ ]:
y_pred = modelo3.predict(X_test)

regression_results(y_test, y_pred)